## Bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from pathlib import Path
import os
pd.options.mode.chained_assignment = None

## Ler base

In [5]:
# Caminho da base
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base' / 'Base Ibovespa Tratada.csv'

# Carrega a base
acao = pd.read_csv(caminho)
acao['Data'] = pd.to_datetime(acao['Data'])

# Ordena por data (caso não esteja)
acao = acao.sort_values(by="Data")

acao

,Data,Último,Abertura,Máxima,Mínima,Variacao (%)
0,2015-01-05,47.517,48.512,48.512,47.264,-0.0205
1,2015-01-06,48.001,47.517,48.061,47.338,0.0102
2,2015-01-07,49.463,48.006,49.882,48.006,0.0305
3,2015-01-08,49.943,49.463,50.261,49.017,0.0097
4,2015-01-09,48.840,49.955,49.955,48.501,-0.0221
...,...,...,...,...,...,...
2577,2025-05-28,138.888,139.541,139.547,138.580,-0.0047
2578,2025-05-29,138.534,138.869,139.108,137.993,-0.0025
2579,2025-05-30,137.027,138.546,138.637,136.726,-0.0109
2580,2025-06-02,136.787,137.026,138.471,136.483,-0.0017


In [7]:
# Define ontem e anteontem
ontem = acao['Data'].max()
anteontem = acao['Data'].iloc[-2]

print(f'Ontem: {ontem}\nAnteontem: {anteontem}')

Ontem: 2025-06-03 00:00:00
Anteontem: 2025-06-02 00:00:00


In [ ]:
# Filtrando da base de treino até anteontem
acao_treino = acao[acao['Data'] < ontem]
cotacao_treino = acao_treino['Último'].to_numpy().reshape(-1, 1)

In [9]:
# Escala os dados
escalador = MinMaxScaler(feature_range=(0, 1))
dados_escalados = escalador.fit_transform(cotacao_treino)

In [10]:
# Cria dados de entrada para o modelo
treinamento_x = []
treinamento_y = []

for i in range(60, len(dados_escalados)):
    treinamento_x.append(dados_escalados[i-60:i, 0])
    treinamento_y.append(dados_escalados[i, 0])

In [11]:
treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

In [12]:
# Define e treina o modelo
modelo = Sequential()
modelo.add(LSTM(100, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
modelo.add(LSTM(50, return_sequences=False))
modelo.add(Dense(25))
modelo.add(Dense(1))
modelo.compile(optimizer="adam", loss="mean_squared_error")
modelo.fit(treinamento_x, treinamento_y, batch_size=10, epochs=10)

Epoch 1/10


c:\Users\YUPOPIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


253/253 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - loss: 0.0310
Epoch 2/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 9.6832e-04
Epoch 3/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 6.9408e-04
Epoch 4/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 7.3466e-04
Epoch 5/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - loss: 4.5575e-04
Epoch 6/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - loss: 3.9659e-04
Epoch 7/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 4.3503e-04
Epoch 8/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 4.5061e-04
Epoch 9/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 3.0250e-04
Epoch 10/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 3.3535e-04


In [13]:
# Pega os últimos 60 dias até ANTEONTEM para previsão
ultimos_60 = dados_escalados[-60:]
entrada_predicao = ultimos_60.reshape(1, 60, 1)

In [21]:
# Faz a previsão para ontem
predicao = modelo.predict(entrada_predicao)
predicao = escalador.inverse_transform(predicao)

# Valor real de ontem
valor_real = acao[acao['Data'] == ontem]['Último'].values[0]

# Exibe os resultados
print(f"Previsão para {ontem.date()}: R$ {predicao[0][0]:}")
print(f"Valor real em {ontem.date()}: R$ {valor_real:}")

# Métricas
erro_abs = abs(valor_real - predicao[0][0])
erro_pct = erro_abs / valor_real * 100
print(f"Erro absoluto: R$ {erro_abs:}")
print(f"Erro percentual: {erro_pct:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Previsão para 2025-06-03: R$ 137.4725341796875
Valor real em 2025-06-03: R$ 136.685
Erro absoluto: R$ 0.7875341796874977
Erro percentual: 0.58%
